*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
def plot_data(dataset):
  #Lets see the age distribution
  print('Age distribution')
  dataset.age.hist(bins=20)
  plt.show()
  #Lets see the sex distribution
  print('Sex distribution')
  dataset.sex.value_counts().plot(kind='barh')
  plt.show()
  #Lets see the bmi distribution
  print('BMI distribution')
  dataset.bmi.hist(bins=20)
  plt.show()
  print('Expenses distribution')
  dataset.expenses.hist(bins=20)
  plt.show()

In [ ]:
print(type(dataset))
print("Size dataset:",dataset.size,'Total Rows:',len(dataset))

In [ ]:
#Categorical data and Numerical data.

CATEGORICAL_COLUMNS = ['sex', 'smoker', 'region']
#print("Values region: ", dataset['region'].unique()) # 4 values.
NUMERIC_COLUMNS = ['age', 'bmi','children']

def categorical_to_numeric(data,columns):
  feature_columns = {}
  for feature_name in columns:
    # gets a list of all unique values from given feature column
    vocabulary = data[feature_name].unique()
    i = 0
    for key in vocabulary:
      feature_columns[key] = i
      i+=1 
  return feature_columns


In [ ]:
#Get dictionary value (categorical) = convertion (numeric)
feature_columns = categorical_to_numeric(dataset,CATEGORICAL_COLUMNS)

print(type(feature_columns),'\n',feature_columns)

In [ ]:
dataset = dataset.replace(feature_columns)
dataset.tail()

In [ ]:
#Split data train 80% - test 20%
def get_dataset_partitions_from_pandas(df, train_split=0.8, val_split=0.1, test_split=0.1):
    assert (train_split + test_split + val_split) == 1
    
    # Seed for same split.
    df_sample = df.sample(frac=1, random_state=10)
    train_index = int(train_split * len(df))
    val_index = int(val_split * len(df)) + train_index
    #Split [train_index,val_index] = train[:train_index], val[train_index:val_index], test[val_index:]
    print(train_index,val_index)
    train_ds, val_ds, test_ds = np.split(df_sample, [train_index,val_index])
    
    return train_ds, val_ds, test_ds

plot_data(dataset)

train_dataset, val_dataset, test_dataset = get_dataset_partitions_from_pandas(dataset,train_split=0.8,val_split=0.0,test_split=0.2)


In [ ]:
print("Train Size:",len(train_dataset),"Validation size:", len(val_dataset), "Test size:",len(test_dataset))

In [ ]:
plot_data(train_dataset)

In [ ]:
plot_data(test_dataset)

In [ ]:
#Pop expenses
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')


In [ ]:
train_dataset.tail()


In [ ]:
test_dataset.tail()

In [ ]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
train_stats

In [ ]:
#Create deep learning model.
normalizer = layers.experimental.preprocessing.Normalization()
normalizer.adapt(np.array(train_dataset))

def build_model():
  model = keras.Sequential([
    normalizer,
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(4, activation='relu'),
    layers.Dense(1)
  ])

  optimizer=tf.optimizers.Adam(learning_rate=0.1)
  model.compile(loss='mae',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

model = build_model()
model.build()
model.summary()

In [ ]:
#it seems its working
example_batch = train_dataset[:10]
example_result = model.predict(example_batch)
example_result

In [ ]:
#Train model

history = model.fit(train_dataset, train_labels, epochs=100, validation_split = 0.1)

In [ ]:
#Plot function. credits: tensorflow.com

def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.legend()
  plt.show()


plot_history(history)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
